In [161]:
import pandas as pd
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [162]:
df = pd.read_parquet('./data/taxi_202101.parquet')

In [163]:
len(df)

1154112

In [164]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime

In [165]:
df['duration'] = df.duration.dt.total_seconds() / 60

In [166]:
df['duration'].mean()

19.167224093791006

In [167]:
df = df[(df.duration >= 1) & (df.duration < 60)]

In [168]:
categorical = ['PUlocationID', 'DOlocationID']
df[categorical] = df[categorical].fillna(-1).astype(int)

In [169]:
df[categorical] = df[categorical].astype(str)

In [170]:
train_dicts = df[categorical].to_dict(orient='records')

In [171]:
train_dicts

[{'PUlocationID': '-1', 'DOlocationID': '-1'},
 {'PUlocationID': '-1', 'DOlocationID': '-1'},
 {'PUlocationID': '-1', 'DOlocationID': '72'},
 {'PUlocationID': '-1', 'DOlocationID': '61'},
 {'PUlocationID': '-1', 'DOlocationID': '71'},
 {'PUlocationID': '-1', 'DOlocationID': '91'},
 {'PUlocationID': '-1', 'DOlocationID': '39'},
 {'PUlocationID': '-1', 'DOlocationID': '37'},
 {'PUlocationID': '-1', 'DOlocationID': '39'},
 {'PUlocationID': '-1', 'DOlocationID': '72'},
 {'PUlocationID': '-1', 'DOlocationID': '72'},
 {'PUlocationID': '-1', 'DOlocationID': '89'},
 {'PUlocationID': '-1', 'DOlocationID': '177'},
 {'PUlocationID': '-1', 'DOlocationID': '225'},
 {'PUlocationID': '-1', 'DOlocationID': '63'},
 {'PUlocationID': '-1', 'DOlocationID': '67'},
 {'PUlocationID': '-1', 'DOlocationID': '22'},
 {'PUlocationID': '-1', 'DOlocationID': '61'},
 {'PUlocationID': '-1', 'DOlocationID': '14'},
 {'PUlocationID': '-1', 'DOlocationID': '14'},
 {'PUlocationID': '-1', 'DOlocationID': '188'},
 {'PUlocat

In [172]:
dv = DictVectorizer()
X_train =dv.fit_transform(train_dicts)

In [173]:
X_train.shape

(1107511, 525)

In [174]:
y_train = df.duration.values

In [176]:
X_train

<1107511x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2215022 stored elements in Compressed Sparse Row format>

In [177]:
len(dv.feature_names_)

525

In [179]:
df.DOlocationID.value_counts()

-1     146976
76      26367
217     19487
265     18489
17      18419
        ...  
27         18
30         13
2          11
105         4
199         1
Name: DOlocationID, Length: 263, dtype: int64

In [180]:
262 + 263

525

In [182]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [183]:
y_pred = lr.predict(X_train)

In [184]:
mean_squared_error(y_train, y_pred, squared=False)

10.393163042792676

In [185]:
categorical = ['PUlocationID', 'DOlocationID']
def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df['duration'].dt.total_seconds() / 60 

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    df[categorical] = df[categorical].fillna(-1).astype(int).astype(str)

    return df

In [186]:
df_val= read_data('./data/taxi_202102.parquet')

In [187]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [188]:
X_val = dv.transform(val_dicts)

In [189]:
y_pred = lr.predict(X_val)

In [190]:
y_val = df_val.duration.values

In [191]:
y_val 

array([10.66666667, 14.56666667,  7.95      , ..., 25.38333333,
       18.05      , 16.        ])

In [192]:
mean_squared_error(y_val, y_pred, squared = False)

11.024909194188771